In [1]:
# IMPORT

from copy import copy

import numpy as np

from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [2]:
# CLASSIFIER

class StepBinaryClassifier(object):
    
    def __init__(self, model=None):
        '''
        Input:
            - model - LogisticRegression / RandomForest / GradientBoosting etc.;
            
        Fields:
            - fitted : bool;
            - classes : list, unique values of y-argument in self.fit;
            - models : list, list of models in classifier;
            - model : model of machine learning, base of this classifier;
            
        Methods:
            - fit(self, X, y);
            - predict(self, X);
            - score(self, X, y);
        '''
        self.model = model
    
        self.fitted = False
        
        self.models = []
        
    def fit(self, X, y):
        '''
        Input:
            - X : np.array, values of features;
            - y : np.array, target;
            
        Returns:
            - self;
        '''
        
        self.classes = list(np.unique(y))       
        self.classes.sort()
        
        for cl in self.classes[:-1]:
            
            true_X = X[y == cl]
            X = X[y != cl]
            
            true_y = [1] * len(true_X)
            false_y = [0] * len(X)
            y = y[y != cl]
            
            new_model = copy(self.model)
            
            fitting_X = np.vstack([true_X, X])
            fitting_y = np.concatenate([true_y, false_y])
            
            new_model.fit(fitting_X, fitting_y)
            self.models.append(new_model)
            
        self.fitted = True
        
        return self
        
    def predict(self, X):
        '''
        Input:
            - X : np.array, values of features, shape=(n, );
        
        Returns:
            - target-class;
        '''
        for i in range(len(self.classes)-1):
            prd = self.models[i].predict([X])
            
            if prd == 1:
                return self.classes[i]
            
        return self.classes[-1]
    
    def score(self, X, y, metric='percent'):
        '''
        Input:
            - X : np.array, values of features, shape=(n, m);
            - y : np.array, right target, shape=(n, );
            - metric : str, 'percent' / 'accuracy', metric of returning value;
        Returns:
            - percent of right predictions (0 - 100%) if metric == 'percent';
            - model's accuracy (0-1) if metric=='accuracy'
            - raises ValueError in other cases; 
        '''
        
        self.predictions = np.array([self.predict(x) for x in X])
        
        acc = np.sum(self.predictions == y) / len(y)
        
        if metric == 'percent':
            return acc * 100
        elif metric == 'accuracy':
            return acc
        else:
            raise ValueError('Argument metric must be \'percent\' or \'accuracy\'.')
            

In [3]:
# TESTING on Iris

model = LogisticRegression()

data = load_iris()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2)

clf = StepBinaryClassifier(model=model).fit(X_train, y_train)
print(clf.score(X_test, y_test))

93.33333333333333


In [6]:
# TESTING on Breast Cancer

model = MLPClassifier(shuffle=False, alpha=0.001)

data = load_breast_cancer()

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.1)

clf = StepBinaryClassifier(model=model).fit(X_train, y_train)
mlp = model.fit(X_train, y_train)
print(clf.score(X_test, y_test, metric='accuracy'))
print(mlp.score(X_test, y_test))

0.9122807017543859
0.8771929824561403
